In [9]:
connect_cmd = "dbname='scielo_db' user='neilor' host='localhost' password='n1f2c3n1'"
%load_ext sql
%sql postgresql://neilor:n1f2c3n1@/scielo_db
import psycopg2 as pg
import os
        

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [5]:
query_create_table_artigos_selecionados = """
drop table if exists alacip.artigos_selecionados;
create table alacip.artigos_selecionados
(
    data_extracao    varchar,
    scielo_id    varchar,
    
    -- PERIODICO
    periodico_classe    varchar,
    periodico_issn_scielo    varchar,
    periodico_cod    varchar,
    periodico_sigla    varchar,
    periodico_nome    varchar,
    periodico_qualis_cp    varchar,
    periodico_area    varchar,
    periodico_instituicao    varchar,
    periodico_cod_instituicao    varchar,
    periodico_cidade_da_instituicao    varchar,
    periodico_cod_cidade    varchar,
    
    -- ARTIGO
    artigo_titulo    varchar,
    artigo_ano    varchar,
    artigo_ano_capes varchar,
    artigo_ano_capes_periodo varchar,
    artigo_tipo    varchar,
    artigo_n_autores    varchar,
    artigo_n_autores_descricao    varchar,
    artigo_n_paginas    varchar,
    artigo_n_referencias   varchar,
    artigo_acessos_ao_html varchar, 
    artigo_acessos_ao_pdf varchar, 
    artigo_area_tematica varchar,
    artigo_link_html varchar,
    
    -- AUTORES
    bolsistas_pq varchar, 
    
    autor_1_nome varchar,
    autor_1_pq  varchar,
    autor_1_vinculo_institucional varchar,
    autor_1_sexo varchar,
    autor_1_docente varchar,

    autor_2_nome varchar,
    autor_2_pq  varchar,
    autor_2_vinculo_institucional varchar,
    autor_2_sexo varchar,
    autor_2_docente varchar,

    autor_3_nome varchar,
    autor_3_pq  varchar,
    autor_3_vinculo_institucional varchar,
    autor_3_sexo varchar,
    autor_3_docente varchar,

    autor_4_nome varchar,
    autor_4_pq  varchar,
    autor_4_vinculo_institucional varchar,
    autor_4_sexo varchar,
    autor_4_docente varchar,

    autor_5_nome varchar,
    autor_5_pq  varchar,
    autor_5_vinculo_institucional varchar,
    autor_5_sexo varchar,
    autor_5_docente varchar,

    autor_6_nome varchar,
    autor_6_pq  varchar,
    autor_6_vinculo_institucional varchar,
    autor_6_sexo varchar,
    autor_6_docente varchar,

    autor_7_nome varchar,
    autor_7_pq  varchar,
    autor_7_vinculo_institucional varchar,
    autor_7_sexo varchar,
    autor_7_docente varchar,

    autor_8_nome varchar,
    autor_8_pq  varchar,
    autor_8_vinculo_institucional varchar,
    autor_8_sexo varchar,
    autor_8_docente varchar,

    autor_9_nome varchar,
    autor_9_pq  varchar,
    autor_9_vinculo_institucional varchar,
    autor_9_sexo varchar,
    autor_9_docente varchar,

    autor_10_nome varchar,
    autor_10_pq  varchar,
    autor_10_vinculo_institucional varchar,
    autor_10_sexo varchar,
    
    -- DADOS XML
    resumo text,
    palavra_chave1 varchar,
    palavra_chave2 varchar,
    palavra_chave3 varchar,
    palavra_chave4 varchar,
    palavra_chave5 varchar,
    empirico varchar,
    tem_tabela varchar,
    tem_quadro varchar,
    tem_grafico varchar,
    tem_diagrama varchar,
    tem_figura varchar,
    abstract text,
    key_word1 varchar,
    key_word2 varchar,
    key_word3 varchar,
    key_word4 varchar,
    key_word5 varchar,
    texto text   
);
"""    
    
    
    
%sql {query_create_table_artigos_selecionados}



Done.
Done.


[]

In [6]:
query_populate_alacip_artigos_selecionados= """
insert into alacip.artigos_selecionados  
(
data_extracao,
scielo_id,
periodico_classe,
periodico_issn_scielo,
periodico_cod,
periodico_sigla,
periodico_nome,
periodico_qualis_cp,
periodico_area,
periodico_instituicao,
periodico_cod_instituicao,
periodico_cidade_da_instituicao,
periodico_cod_cidade,
artigo_titulo,
artigo_ano,
artigo_ano_capes,
artigo_ano_capes_periodo,
artigo_tipo,
artigo_n_autores,
artigo_n_autores_descricao,
artigo_n_paginas,
artigo_n_referencias,
artigo_link_html
)
select 

d.document_extraction_date    as    data_extracao,
d.document_publishing_id_scielo    as    scielo_id,
      -- PERIODICO
p.periodico_classe,
p.periodico_issn_scielo,
p.periodico_cod,
p.periodico_sigla,
p.periodico_nome,
p.periodico_qualis_cp,
p.periodico_area,
p.periodico_instituicao,
p.periodico_cod_instituicao,
p.periodico_cidade_da_instituicao,
p.periodico_cod_cidade,
      -- ARTIGO
'#NONE'    as    artigo_titulo,
d.document_publishing_year    as    artigo_ano,
'#NONE' as artigo_ano_capes,
'#NONE' as artigo_ano_capes_periodo,
d.document_type    as    artigo_tipo,
d.document_authors    as    artigo_n_autores,
'#NONE'    as    artigo_n_autores_descricao,
document_pages    as    artigo_n_paginas,
document_referencias    as    artigo_n_referencias,
'#NONE' as artigo_link_html


FROM 
  alacip.periodicos_selecionados as p
left join
  public.documents_counts as d
on
  d.document_publishing_year in 
    (
    '2013','2014','2015','2016'
    ,'2010','2011','2012'
    ,'2007','2008','2009'
    ,'2004','2005','2006'
    ,'2001','2002','2003'
    ,'1998','1999','2000'
    ,'1995','1996','1997'
    ) 
AND 
  d.document_issn_scielo = p.periodico_issn_scielo
AND
  d.document_type like 'research-article'
;

create index scielo_idx on alacip.artigos_selecionados(scielo_id);
"""

%sql {query_populate_alacip_artigos_selecionados}


7233 rows affected.
Done.


[]

In [7]:
query_update_ano_capes = """
update alacip.artigos_selecionados
set artigo_ano_capes =
  CASE WHEN artigo_ano in ('2013','2014','2015','2016') THEN '1'
       WHEN artigo_ano in ('2010','2011','2012') THEN '2'
       WHEN artigo_ano in ('2007','2008','2009') THEN '3' 
       WHEN artigo_ano in ('2004','2005','2006') THEN '4'
       WHEN artigo_ano in ('2001','2002','2003') THEN '5'
       WHEN artigo_ano in ('1998','1999','2000') THEN '6'  
       WHEN artigo_ano in ('1995','1996','1997') THEN '7'  
  END
;
"""

%sql {query_update_ano_capes}

query_update_ano_capes_periodo = """
update alacip.artigos_selecionados
set artigo_ano_capes_periodo =
  CASE WHEN artigo_ano in ('2013','2014','2015','2016') THEN '2013,2014,2015,2016'
       WHEN artigo_ano in ('2010','2011','2012') THEN '2010,2011,2012'
       WHEN artigo_ano in ('2007','2008','2009') THEN '2007,2008,2009' 
       WHEN artigo_ano in ('2004','2005','2006') THEN '2004,2005,2006'
       WHEN artigo_ano in ('2001','2002','2003') THEN '2001,2002,2003'
       WHEN artigo_ano in ('1998','1999','2000') THEN '1998,1999,2000'  
       WHEN artigo_ano in ('1995','1996','1997') THEN '1995,1996,1997'  
  END
;
"""

%sql {query_update_ano_capes_periodo}

7233 rows affected.
7233 rows affected.


[]

In [8]:
query_update_n_autores_descricao = """
update alacip.artigos_selecionados
set artigo_n_autores_descricao =
  CASE WHEN artigo_n_autores like '1' THEN 'um autor'
       WHEN artigo_n_autores like '2' THEN 'dois autores' 
       WHEN artigo_n_autores like '3' THEN 'três autores'
       WHEN artigo_n_autores like '4' THEN 'quatro autores'
       WHEN artigo_n_autores like '5' THEN 'cinco autores'
       WHEN artigo_n_autores like '6' THEN 'seis autores'
       WHEN artigo_n_autores like '7' THEN 'sete autores'
       WHEN artigo_n_autores like '8' THEN 'oito autores'
       WHEN artigo_n_autores like '9' THEN 'nove autores'
       WHEN artigo_n_autores like '10' THEN 'dez autores'
       WHEN artigo_n_autores like null  THEN 'não definido'
       WHEN artigo_n_autores like ' '  THEN 'não definido'
       WHEN artigo_n_autores like '0'  THEN 'não definido'
       ELSE 'mais de dez autores'
  END
;
"""

%sql{query_update_n_autores_descricao}

print("OK")

7233 rows affected.
OK


In [9]:
query_create_coluna_link_html = """
update alacip.artigos_selecionados
set artigo_link_html = 'http://www.scielo.br/scielo.php?script=sci_arttext&pid='||scielo_id
;
"""

%sql {query_create_coluna_link_html}

7233 rows affected.


[]